#LoRa Signal Classification

In [156]:
MODEL_INPUT_SHAPE = (1024, 1)
MODEL_OUTPUT_SHAPE = (18,)

F_SAMP = int(1e6)
F_MAX = 250000
WINDOW_SIZE = 2**7
WINDOW_OVERLAP = WINDOW_SIZE // 2

BW_VALUES = [125000, 250000, 500000]
SF_VALUES = [7, 8, 9, 10, 11, 12]

SNR_MIN = -15
SNR_MAX = 20
SNR_STEP = 2
PATH_RESULTS = '/content/Results'
PATH_PREAMBLE_REFERENCE_IQ = '/content/Preamble_symbols'

PATH_TRAIN_DATASET = '/content/Train_Dataset'
PATH_VAL_DATASET = '/content/Validation_Dataset'
PATH_FIGURES = '/content/Figures'
FONT_SIZE = 16
FONT_SIZE_LEGEND = 12
PATH_FIGURES = '/content/Figures'

Imports

In [157]:
import tensorflow as tf
from tensorflow.keras import models
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Flatten, Dense, Dropout
from sklearn.model_selection import train_test_split
from scipy.signal import spectrogram
import matplotlib.pyplot as plt
import numpy as np
import os
from scipy.stats import t

Utility

In [ ]:
def encode_label(config):
  one_hot_label = np.zeros(18)
  one_hot_label[config] = 1
  return one_hot_label

Dataset

In [158]:
def read_iq_file(filepath):
    with open(filepath, "rb") as f:
        iq_samples = np.fromfile(f, dtype=np.complex64)
    return iq_samples

def read_IF_sample(file_path):
  IF_sample = np.load(file_path)[-MODEL_INPUT_SHAPE[0]:]
  return IF_sample

def get_reference_iq_samples(bandwidth, spread_factor):
  iq_file_path = os.path.join(PATH_PREAMBLE_REFERENCE_IQ,
                                  'PR_Ref_BW{}_SF{}.bin'.format(
                                      BW_VALUES.index(bandwidth),
                                      spread_factor))
  return read_iq_file(iq_file_path)

def pad_signal(signal_IQ):
  signal_len = signal_IQ.shape[0]
  input_length= (MODEL_INPUT_SHAPE[0]+1)*WINDOW_OVERLAP

  padded_signal = np.zeros(input_length, dtype= np.complex64)
  if signal_len <= input_length:
    padded_signal[-signal_len:] = signal_IQ
  else:
    padded_signal = signal_IQ[-input_length:]
  return padded_signal

def add_gaussian_noise(signal, snr_db):
    non_zero_samples = np.nonzero(signal)[0]
    first_non_zero = non_zero_samples[0] if len(non_zero_samples) > 0 else 0
    last_non_zero = non_zero_samples[-1] if non_zero_samples[-1] < len(
        signal) else len(signal)-1
    signal_power = np.mean(np.abs(signal[first_non_zero:last_non_zero+1]) ** 2)
    snr = 10 ** (snr_db / 10.0)
    noise_power = signal_power / snr
    noise = np.sqrt(noise_power / 2) * (np.random.randn(*signal.shape) + 1j * np.random.randn(*signal.shape))
    noisy_signal = signal + noise
    return noisy_signal

def simulate_iq_samples(bandwidth, spread_factor, snr_db):
    IQ_ref = get_reference_iq_samples(bandwidth, spread_factor)
    IQ_noisy = add_gaussian_noise(pad_signal(IQ_ref), snr_db)
    return IQ_noisy

def compute_spectrogram(IQ_samples):
  Sxx =  spectrogram(IQ_samples, F_SAMP,window= 'hann',
                             nperseg= WINDOW_SIZE,noverlap= WINDOW_OVERLAP,
                             return_onesided= False)[2]
  Sxx = np.vstack([Sxx[Sxx.shape[0]//2:], Sxx[:Sxx.shape[0]//2]])
  Sxx = Sxx[WINDOW_SIZE//4:-WINDOW_SIZE//4, :]
  Sxx = (Sxx - np.min(Sxx))/(np.max(Sxx)-np.min(Sxx))
  return Sxx

def compute_inst_freq(spec_sample):
  spec_sample = spec_sample**4
  f_bins = np.linspace(-F_MAX, F_MAX, num=spec_sample.shape[0]+1)[:-1]
  weighted_sum = np.sum(spec_sample * f_bins[:, np.newaxis], axis=0)
  total_power = np.sum(spec_sample, axis=0)
  IF_sample = np.clip((weighted_sum / total_power)/F_MAX, -1, 1)[:, np.newaxis]
  return IF_sample

def simulate_IF_sample(config, snr):
  bandwidth, spread_factor = BW_VALUES[config//6], SF_VALUES[config%6]
  IQ_samples = simulate_iq_samples(bandwidth, spread_factor, snr)
  spec_sample = compute_spectrogram(IQ_samples)
  IF_sample = compute_inst_freq(spec_sample)[-MODEL_INPUT_SHAPE[0]:]
  return IF_sample

def create_dataset(path, snr_range, count):
  for snr in snr_range:
    for config in range(18):
      for i in range(count):
        file_name = f'SNR{encode_snr(snr)}C{config:02}T{i:02}.npy'
        file_path = os.path.join(path, file_name)
        IF_sample = simulate_IF_sample(config, snr)
        np.save(file_path, IF_sample)

In [ ]:
def encode_snr(snr):
    if snr < 0:
        snr_sign = 1
        snr_mag = int(abs(snr) * 100)
    else:
        snr_sign = 0
        snr_mag = int(snr * 100)
    encoded_snr = snr_sign * 10000 + snr_mag
    return f"{encoded_snr:05}"

def decode_snr(snr):
  # Decode SNR from file name (sign: 0/1, magnitude: 0.00-99.99)
  snr_sign, snr_mag = divmod(int(snr), 10000)
  snr = (-1)*(snr_mag/100) if snr_sign else snr_mag/100
  return snr

def parse_file_name(file_path):
  #filename format 'SNR00000C00T00.npy'
  file_name = os.path.basename(file_path)
  match = re.search(r'SNR(\d{5})C(\d{2})T(\d{2}).npy', file_name)
  if match:
      snr = decode_snr(match.group(1))
      config = int(match.group(2))
      index = int(match.group(3))
      return snr, config, index

Model

In [ ]:
def create_model():
  model = Sequential([
      Flatten(input_shape=MODEL_INPUT_SHAPE),
      Dense(16, activation='tanh'),
      Dense(16, activation='tanh'),
      Dropout(0.5),
      Dense(18, activation='softmax')
  ])
  model.compile(loss=tf.keras.losses.categorical_crossentropy,
                optimizer=tf.keras.optimizers.Adam(),
                metrics=['accuracy'])
  return model

def load_model(model_name):
  path_model = os.path.join(PATH_MODELS, model_name)
  return tf.keras.models.load_model(path_model)

def save_model(model, model_name):
  path_model = os.path.join(PATH_MODELS, model_name)
  model.save(path_model)

Model Training

In [159]:
def data_generator(files_list, batch_size):
  while True:
    X, y = [], []
    sample_count = 0
    for file_path in files_list:
      sample = read_IF_sample(file_path)
      config = parse_file_name(os.path.basename(file_path))[1]
      label = encode_label(config)
      X.append(sample)
      y.append(label)
      sample_count += 1
      if sample_count == batch_size:
          yield np.array(X), np.array(y)
          X, y = [], []
          sample_count = 0

def train_model_from_files(model, train_data, validation_data):
  batch_size = 8
  train_data_gen = data_generator(train_data, batch_size)
  val_data_gen = data_generator(validation_data, batch_size)
  model.fit(train_data_gen, epochs=5, steps_per_epoch=len(train_data)//batch_size,
            validation_data=val_data_gen, validation_steps=len(validation_data)//batch_size,
            verbose=1)
  return model
def evaluate_model(model, path_val_dataset=PATH_VAL_DATASET, snr_range=list(range(SNR_MIN, SNR_MAX, SNR_STEP)), size = 20):
  results_config = []
  for config in range(18):
    results_snr = []
    for snr in snr_range:
      X = []
      y = np.array([encode_label(config)]*size)
      for i in range(size):
        file_name = f'SNR{encode_snr(snr)}C{config:02}T{i:02}.npy'
        sample = read_IF_sample(os.path.join(path_val_dataset, file_name))
        X.append(sample)
      X = np.array(X)
      acc = model.evaluate(X,y)[1]
      results_snr.append(acc)
    results_config.append(np.array(results_snr))
  return np.array(results_config)

Plotting

In [160]:
def select_configs(results, config_choice):
    return results[:, config_choice, :].reshape(-1, results.shape[-1])

def plot_results(results, snr_range, label=None, cf = 0.95):
    n = results.shape[0]
    means = np.mean(results, axis=0)
    stds = np.std(results, axis=0)
    h = stds * t.ppf((1 + cf) / 2, n - 1) / np.sqrt(n)
    cis = np.vstack((means - h, means + h))
    plt.plot(snr_range, means, 'o-', markersize= 3, label=label)
    plt.fill_between(snr_range, cis[0, :], cis[1, :], alpha=0.2)#, label=f'Confidence {int(cf*100)}%')

def display_results_BW(results, snr_range, path_save=None):

    BW_values = [125, 250, 500]
    plt.figure()
    plt.rc('font', size=FONT_SIZE)
    plt.grid()
    plt.ylim(0, 100)
    plt.yticks(range(0, 101, 10), fontsize=FONT_SIZE_LEGEND)
    plt.xticks(snr_range[::2], fontsize=FONT_SIZE_LEGEND)
    plt.title('(3) Accuracy by Bandwidth', fontsize=FONT_SIZE)
    plt.xlabel('Signal to Noise Ratio (dB)')
    plt.ylabel('Accuracy (%)')
    for idx, bw in enumerate(BW_values):
        config_choice = [idx*6+i for i in range(6)]
        reshaped_results = select_configs(results, config_choice)
        plot_results(reshaped_results, snr_range, label=f'BW {bw} kHz')
    plt.legend(loc='lower right', fontsize=FONT_SIZE_LEGEND)
    if path_save is not None:
        plt.savefig(os.path.join(path_save, 'fig_acc_bw.pdf'))
    plt.show()

def display_results_SF(results, snr_range, path_save=None):

    SF_values = [7, 8, 9, 10, 11, 12]
    plt.figure()
    plt.rc('font', size=FONT_SIZE)
    plt.grid()
    plt.ylim(0, 100)
    plt.yticks(range(0, 101, 10), fontsize=FONT_SIZE_LEGEND)
    plt.xticks(snr_range[::2], fontsize=FONT_SIZE_LEGEND)
    plt.title('(2) Accuracy by Spreading Factor', fontsize=FONT_SIZE)
    plt.xlabel('Signal to Noise Ratio (dB)')
    plt.ylabel('Accuracy (%)')
    for idx, sf in enumerate(SF_values):
        config_choice = [i*6+idx for i in range(3)]
        reshaped_results = select_configs(results, config_choice)
        plot_results(reshaped_results, snr_range, label=f'SF {sf}')
    plt.legend(loc='lower right', fontsize=FONT_SIZE_LEGEND)
    if path_save is not None:
        plt.savefig(os.path.join(path_save, 'fig_acc_sf.pdf'))
    plt.show()

def display_results(results, snr_range, path_save=None):

    plt.figure()
    plt.rc('font', size=FONT_SIZE)
    plt.grid()
    plt.ylim(0, 100)
    plt.xlim(-16, 22)
    plt.yticks(range(0, 101, 10), fontsize=FONT_SIZE_LEGEND)
    plt.xticks(snr_range[::2], fontsize=FONT_SIZE_LEGEND)
    plt.title('(1) Overall Accuracy', fontsize=FONT_SIZE)
    plt.xlabel('Signal to Noise Ratio (dB)')
    plt.ylabel('Accuracy (%)')
    results = results.reshape(-1, results.shape[-1])
    plot_results(results, snr_range, label='Mean Accuracy')
    plt.legend(loc='lower right', fontsize=FONT_SIZE_LEGEND)
    if path_save is not None:
        plt.savefig(os.path.join(path_save, 'fig_acc.pdf'))
    plt.show()


In [161]:
def display_all_results(results, path=PATH_FIGURES):
  snr_range = np.arange(SNR_MIN, SNR_MAX, SNR_STEP)
  acc_results =results*100
  display_results(acc_results, snr_range, path_save=path)
  display_results_SF(acc_results, snr_range, path_save=path)
  display_results_BW(acc_results, snr_range, path_save=path)

Simulation

In [ ]:
def run_simulation():
  train_dataset = [os.path.join(PATH_TRAIN_DATASET, x) for x in os.listdir(PATH_TRAIN_DATASET) if os.path.isfile(os.path.join(PATH_TRAIN_DATASET, x))]
  num_iterations = 30
  results = []
  snr_range = list(range(SNR_MIN, SNR_MAX+1, SNR_STEP))
  for i in range(num_iterations):
    print(f'Iteration {i+1}')
    train_files, test_files = train_test_split(train_dataset, test_size=0.2,
                                               shuffle=True, random_state=40+i)
    model = create_model()
    model = train_model_from_files(model, train_data= train_files,
                                   validation_data= test_files)
    results_i = evaluate_model(model, PATH_VAL_DATASET, snr_range)
    results.append(results_i)
  print('Simulation end!')
  return np.array(results)


In [ ]:
results = run_simulation()
np.save(os.path.join(PATH_RESULTS, 'simulation_results.npy'), results)
display_all_results(results)